# Práctico de Exploración y Curación

<font size=5>Reducción de Emisiones Contaminantes por el Uso de Biocombustibles en Transporte de Cargas y Pasajeros</font>

<font size=4>Mentoria DiploDatos FAMAF</font>

In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import zipfile

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 50)

## Importando datos
_Datasets :_
- Productos: registra datos de los tipos de combustibles
- Bombas: registra datos de las bombas de suministro de combustible
- Equipos: registra datos de los equipos IoT
- Tanques: registra datos de los tanques de almacenamiento
- Usuarios: registra datos de los usuarios de las bombas
- Vehiculos: registra datos de los vehículos 
- Historial (mensuales, desde el 2018): registro de los inventarios mensuales de los tanques de combustibles
- Transacciones (mensuales, desde el 2018): registro de los suministros de combustible por cada bomba a cada vehículo

In [3]:
# pip install memory_profiler
%load_ext memory_profiler

In [4]:
%run functions.py

### Descompresión y lectura de csv

In [5]:
%%time
%memit
zf = zipfile.ZipFile('./data/data_CTL.zip')
files = zf.infolist()

print ("Uncompressing and reading data...\n")

for idx, file in enumerate(files):
    name_file = (file.filename).split('/')[1][:-4]
    print(f'Cargando...\nDataframe {idx+1} de {len(files)}: {name_file}\n')
    vars()[name_file] = pd.read_csv(zf.open(file.filename))
    vars()[name_file].name = name_file
print('Proceso finalizado')

peak memory: 123.10 MiB, increment: 0.45 MiB
Uncompressing and reading data...

Cargando...
Dataframe 1 de 17: fs_asignacion_producto

Cargando...
Dataframe 2 de 17: fs_bombas

Cargando...
Dataframe 3 de 17: fs_equipo

Cargando...
Dataframe 4 de 17: fs_tanques

Cargando...
Dataframe 5 de 17: fs_usuarios_fuelsentry

Cargando...
Dataframe 6 de 17: fs_vehiculos

Cargando...
Dataframe 7 de 17: sis_historial_S1_2018

Cargando...
Dataframe 8 de 17: sis_historial_S1_2020

Cargando...
Dataframe 9 de 17: sis_historial_S2_2018

Cargando...
Dataframe 10 de 17: sis_historial_S2_2019

Cargando...
Dataframe 11 de 17: sis_historial_T1_2019

Cargando...
Dataframe 12 de 17: sis_historial_T2_2019

Cargando...
Dataframe 13 de 17: sis_transa_S1_2018

Cargando...
Dataframe 14 de 17: sis_transa_S1_2019

Cargando...
Dataframe 15 de 17: sis_transa_S1_2020

Cargando...
Dataframe 16 de 17: sis_transa_S2_2018

Cargando...
Dataframe 17 de 17: sis_transa_S2_2019

Proceso finalizado
Wall time: 2min 56s


### Armado de dataframe de transacciones (2018-2020)

In [6]:
%%time
%memit
lista_dataframes_varios = [fs_bombas, fs_equipo, fs_tanques, fs_usuarios_fuelsentry, fs_vehiculos]
lista_dataframes_transacc = [sis_transa_S1_2018, sis_transa_S2_2018, sis_transa_S1_2019, sis_transa_S2_2019, sis_transa_S1_2020]
lista_dataframes_historial = [sis_historial_S1_2018, sis_historial_S2_2018, sis_historial_T1_2019, sis_historial_T2_2019, sis_historial_S2_2019, sis_historial_S1_2020]

df_transacciones = pd.concat([df for df in lista_dataframes_transacc], ignore_index=True)
df_transacciones.name = 'Transacciones (2018-2020)'
df_transacciones.head()

peak memory: 12049.34 MiB, increment: 0.59 MiB
Wall time: 9 s


,id_transaccion,id_vehiculo,id_usuario,id_equipo,producto,id_bomba,id_tanque,departamento,fecha,hora,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,codigo_error,valor,campovar1,campovar2,baja,fecha_stop,hora_stop,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,geo_status,veh_efficiency
0,6974,39920,1045,139,1.0,174,134,0000,2018-01-01,0 days 08:27:00.000000000,115.972,30665.8,99999.4,000012,0000,BF,0.0,0000,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6975,39819,1041,139,1.0,174,134,0000,2018-01-01,0 days 13:56:00.000000000,127.907,4723.7,79600.4,435885,0531,BF,0.0,0000,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6976,39931,1055,139,1.0,174,134,0000,2018-01-01,0 days 16:54:00.000000000,65.571,4324,42312.1,000000,0000,BF,0.0,0000,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6977,39922,1047,139,1.0,174,134,0000,2018-01-01,0 days 20:38:00.000000000,71.701,2506.3,14088.7,000000,0000,BF,0.0,0000,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5250,41504,436,111,1.0,100,81,0001,2018-01-01,0 days 00:23:00.000000000,360.018,124,296,464294,0000,C1,0.0,0000,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Armado de dataframe de historiales (2018-2020)

In [7]:
%%time
%memit
df_historiales = pd.concat([df for df in lista_dataframes_historial], ignore_index=True)
df_historiales.name = 'Historiales (2018-2020)'
df_historiales.head()

peak memory: 13003.83 MiB, increment: 0.00 MiB
Wall time: 1min 11s


,id,id_equipo,id_tanque,fecha,hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,60147,479,1037,2018-01-01,0 days 05:22:00.000000000,32602.2,20.0,V0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60148,479,1037,2018-01-01,0 days 06:35:00.000000000,32515.1,21.0,V1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60149,479,1037,2018-01-01,0 days 07:30:00.000000000,32602.2,20.0,N0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60150,479,1037,2018-01-01,0 days 08:07:00.000000000,32576.6,19.0,V0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60151,479,1037,2018-01-01,0 days 08:30:00.000000000,32530.3,21.0,N1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Revisando tipos de datos, valores nulos y dimensiones de los dataframes

In [8]:
df_transacciones.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820449 entries, 0 to 2820448
Data columns (total 31 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id_transaccion  2820449 non-null  int64  
 1   id_vehiculo     2820449 non-null  int64  
 2   id_usuario      2820449 non-null  int64  
 3   id_equipo       2820449 non-null  int64  
 4   producto        2810641 non-null  float64
 5   id_bomba        2820449 non-null  int64  
 6   id_tanque       2820449 non-null  int64  
 7   departamento    2820436 non-null  object 
 8   fecha           2820449 non-null  object 
 9   hora            2820449 non-null  object 
 10  cantidad        2820448 non-null  float64
 11  acum_vehiculo   2820425 non-null  object 
 12  acum_usuario    2820429 non-null  object 
 13  odometro        2789270 non-null  object 
 14  km_transaccion  2787975 non-null  object 
 15  codigo_error    2820448 non-null  object 
 16  valor           2820449 non-null  fl

Veamos los tipos de datos y dimensión del dataframe de historiales

In [9]:
# Hacer lo mismo para el dataframe de historiales (si .info() arroja error dejo abajo otra forma de acceder a la info --> remover # y usar)


In [10]:
# Tipos de datos y dimensión del dataframe (a diferencia del .info() acá se mostrará la cantidad de valores nulos)
# shape = df_transacciones.shape
# print('Número de registros: {0} - Número de columnas: {1}'.format(shape[0], shape[1]))
# print('#\tColumn\t\t\tNon-Null Count\t\t\tDtype:\n-----------------------------------------------------------------------')
# for idx, col in enumerate(df_transacciones.columns):
#     print(f'{idx}\t{col: <26}{df_transacciones[col].isnull().sum(): <6} null\t\t\t{df_transacciones[col].dtypes}')

In [11]:
for df in lista_dataframes_varios + [df_transacciones, df_historiales]:
    print('----------------------------------------------------------------------------------------------------------------------------------')
    print(f'\tDataset {df.name}:\n\nNúmero de categorias o valores únicos de cada columna:')
    [print(f'{col: <20}: {df[col].nunique()}') for col in df.columns]

----------------------------------------------------------------------------------------------------------------------------------
	Dataset fs_bombas:

Número de categorias o valores únicos de cada columna:
id_bomba            : 1438
id_equipo           : 610
bomba               : 26
producto            : 10
id_tanque           : 861
totalizador         : 869
fecha_reinicio      : 180
pulsos_litro        : 231
tiempo_interrump    : 25
habilitacion        : 7
rampa_de_parada     : 7
----------------------------------------------------------------------------------------------------------------------------------
	Dataset fs_equipo:

Número de categorias o valores únicos de cada columna:
id_equipo           : 687
id_empresa          : 298
current_firmware    : 67
delay_ue            : 2
geo_latitude        : 17
geo_longitude       : 17
----------------------------------------------------------------------------------------------------------------------------------
	Dataset fs_tanques:

Nú

<font size=4>Dataset de asignación de productos:</font>

Contiene información relacionada con los productos de combustible.

Descripción de las columnas:
- 'id_equipo': ID del dispositivo IoT
- 'producto': código del producto
- 'nombre_producto': nombre del producto
- 'codigo': 
- 'precio_litro': precio del producto (en $/l)
- 'coef_var_vol': coeficiente de variación del volumen del producto combustible (en g/cm3/°C)
- 'density': densidad del producto (en g/cm3)

Densidad (ASTM D 4052): densidad relativa del combustible medido a la temperatura estándar de 15 °C.

In [12]:
fs_asignacion_producto.head()

,id_equipo,producto,nombre_producto,codigo,precio_litro,coef_var_vol,density
0,333333,0,gas oil,gas oil,7.5,0.001,NaN
1,333333,1,Product 2,Product 2,0.0,0.001,NaN
2,333333,2,Product 3,Product 3,0.0,0.001,NaN
3,333333,3,Product 4,Product 4,0.0,0.001,NaN
4,1,0,Diesel,Diesel,1.0,0.001,NaN


<font size=4>Dataset de bombas:</font>

Contiene información de las bombas de suministro de combustible.

--> Un punto de suministro del combustible posee uno o varios equipos (dispositivo IoT) 

Descripción de las columnas:
- 'id_bomba': ID de la bomba
- 'id_equipo': ID del dispositivo IoT
- 'bomba': 
- 'producto': tipo de combustible que suministra la bomba
- 'id_tanque': ID del tanque
- 'totalizador': litros suministrados a la fecha de reinicio
- 'fecha_reinicio': fecha de reinicio de la bomba
- 'pulsos_litro': litros que suministra la bomba (por pulso)
- 'tiempo_interrump': tiempo de interrupción del suministro de combustible de la bomba (en segundos)
- 'habilitacion': 
- 'rampa_de_parada'

In [13]:
fs_bombas.head()

,id_bomba,id_equipo,bomba,producto,id_tanque,totalizador,fecha_reinicio,pulsos_litro,tiempo_interrump,habilitacion,rampa_de_parada
0,1,333333,1,0,1,31731.441,2012-07-19,0.0000,2.0,1,0
1,2,333333,2,0,1,1160601.525,2012-07-19,0.0000,2.0,1,0
2,11417,770,2,1,101856,0.000,2018-07-05,26.3158,20.0,1,0
3,5,444444,1,2,4,651995.027,2010-12-01,35.0000,888.0,1,0
4,6,444444,2,2,4,5232.793,2010-12-01,35.5000,2.0,1,0


In [14]:
fs_equipo.head()

,id_equipo,id_empresa,current_firmware,delay_ue,geo_latitude,geo_longitude
0,585,25,V60403bQ4830361,0,NaN,NaN
1,749,210,V80604bQ0428b17,0,NaN,NaN
2,584,1000,V60403bQ4830361,0,NaN,NaN
3,581,1025,V60403bQ4830361,0,-38.375066,-68.622686
4,599,1050,V60403bQ59fd4f2,180,-31.528835,-68.514707


<font size=4>Dataset de tanques:</font>

Contiene información de los tanques de combustible.

--> Una empresa puede tener más de un tanque

--> Varios tanques pueden estar conectado a un mismo equipo

--> Un tanque posee una o varias bombas

Descripción de las columnas:
- 'id_tanque': ID del tanque
- 'id_equipo': ID del dispositivo IoT 
- 'tanque': tipo de tanque
- 'producto': producto almacenado en el tanque 
- 'capacidad': capacidad del tanque en litros
- 'log_interval': intervalo de tiempo en el cual se registra la medición del volumen del contenido del tanque --> historial del volumen del tanque
- 'nivel_alarma': nivel de contenido del tanque para disparar alarma

In [15]:
fs_tanques.head()

,id_tanque,id_equipo,tanque,producto,capacidad,log_interval,nivel_alarma
0,101525,669,B,1,0010000.1,0030,0.0
1,101833,764,5,1,0008000.0,0030,3200.0
2,101496,662,1,0,0050000.0,0030,0.0
3,101495,661,1,0,0025000.0,0030,0.0
4,101522,669,8,1,0010000.1,0030,0.0


<font size=4>Dataset de usuarios:</font>

Contiene información de los usuarios de las bombas de combustible.

Descripción de las columnas:
- 'id_usuario_fuelsentry': ID de registro del usuario
- 'id_equipo':  ID del dispositivo IoT
- 'usuario_fuelsentry': código del usuario de la bomba
- 'departamento': 
- 'codigo': 
- 'totalizador': litros totales suministrados por el usuario de la bomba
- 'cargas_totales': número total de cargas sumnistradas por el usuario

In [16]:
fs_usuarios_fuelsentry.head()

,id_usuario_fuelsentry,id_equipo,usuario_fuelsentry,departamento,codigo,totalizador,cargas_totales
0,2,333333,0001,0000,0,323.152,1
1,3,333333,1315,0001,2012,1655.590,9
2,4,333333,1312,0001,4468,1433.429,6
3,5,333333,1298,0001,2609,412.834,2
4,6,333333,0818,0001,1978,713.023,3


In [17]:
fs_vehiculos.head()

,id_vehiculo,id_equipo,vehiculo,departamento,limite,odometro_inicio,odometro_fin,cargas_max_dia,autorizacion,cantidad_total,cargas_hoy,cargas_hasta_hoy,ultima_fecha
0,2,333333,MDAwMDAx,0001,9,193958,193958,99,0,00000.000,01,0.0,2018-12-20
1,3,333333,MDAwMjM5,0001,9,079345,079345,99,0,00000.000,02,0.0,2018-12-20
2,4,333333,MDAwMjcw,0001,9,842409,842409,99,0,00000.000,01,0.0,2018-12-20
3,5,333333,MDAwMjUy,0001,9,287596,287596,99,0,00000.000,02,0.0,2018-12-20
4,6,333333,MDAwMjM1,0001,9,235878,235878,99,0,00000.000,01,0.0,2018-12-20


## Curación de datos de historial y transacciones

### Conversion de tipos de datos

Para cada proyecto, se debe conocer las variables de entrada y su naturaleza. Para tomar una decisión respecto del más adecuado tipo de dato que debe tener cada una de las variables, necesitamos responder a las siguientes interrogantes:
- ¿La variable es de tipo categórica o númerica?
- ¿Qué tipo de precisión necesita mi salida? 
- ¿La velocidad es un problema? 
- ¿Qué precisión se necesita en partes por millón?
- ¿Sería necesario almacenar el flotador 32 o el flotador 64?
- Qué conviene ¿Object o categorical?

Con la idea de aminorar el uso de memoria en las ejecuciones que hagamos, sería de gran utilidad preguntarse cómo podemos lograr tener dataframes "más ligeros". Una manera de lograr esto es utilizando un tipo de dato apropiado, tanto para la naturaleza de las variables que tenemos, como para los cálculos que queremos realizar. 

"float32" es un número de 32 bits; "float64" usa 64 bits. Eso significa que float64 ocupa el doble de memoria, y realizar operaciones en ellos puede ser mucho más lento en algunas arquitecturas de máquinas. Pandas por defecto usa el float e int de 64 bits. 

Sin embargo, float64 tiene la capacidad de representar números con mucha más precisión que los flotantes de 32 bits. También permiten almacenar números mucho más grandes. 

También podemos emplear el tipo "categorical" de pandas para economizar el uso de memoria, recuerda que existen las categorias nominales y ordinales.

En función de esto, realizar las respectivas conversiones en los tipos de datos que consideres necesarios. Explique el por qué de su decisión.

Recuerda hacer la conversión correspondiente para datos de fechas y horas.

### Verificación de registros/id duplicados

### Detección e imputación/eliminación de valores perdidos

### Eliminación de columnas

### Revisar incongruencia en valores 
(revisar las fechas y otras columnas que creas conveniente)

### Codificación de variables categóricas (de ser necesario)

### Revisar valores únicos de las variables que considere necesarias

### Realizar las transformaciones que considere importantes

<font size=4 color='blue'>“Data is a precious thing and will last longer than the systems themselves.” – Tim Berners-Lee</font>

- ¿Cuántas bombas tenemos en los datos?
- ¿Cuántas equipo IoT?
- ¿Cuántos tanques?
- ¿Cuántos producto y cuáles serán los seleccionados como objeto de estudio?
- 